In [1]:
import pandas as pd

import requests
import json

from config import gkey

# Extract

In [2]:
file1 = "mass_shooting_2019.csv"
file2 = "mass_shooting_2020.csv"
file3 = "mass_shooting_2021.csv"

In [3]:
df_2019 = pd.read_csv(file1)
df_2020 = pd.read_csv(file2)
df_2021 = pd.read_csv(file3)

# Transform

In [4]:
frames = [df_2019, df_2020, df_2021]

result = pd.concat(frames)

In [5]:
result['Address'] = result['Address'].str.replace("block of", '')

In [6]:
result["Full_Address"] = result['Address'] + ', ' + result['City Or County'] + ' ' + result['State']

In [7]:
result = result.drop(['Operations'], axis=1)


result = result.rename(columns={"Incident ID": "incident_id"
                   , "Incident Date": "incident_date"
                   , "State": "state"
                   , "City Or County": "city_county"
                   , "Address": "street_address"
                   , "# Killed": "killed"
                   , "# Injured": "injured"
                  , "Full_Address": "full_address"})

In [8]:
result.head()

,incident_id,incident_date,state,city_county,street_address,killed,injured,full_address
0,1582816,"December 29, 2019",Illinois,Danville,1803 Westview Ave,0,5,"1803 Westview Ave, Danville Illinois"
1,1582197,"December 29, 2019",New York,Buffalo,50 Henrietta Ave,1,3,"50 Henrietta Ave, Buffalo New York"
2,1583444,"December 29, 2019",California,Ceres,800 Allacante Dr,0,5,"800 Allacante Dr, Ceres California"
3,1581590,"December 27, 2019",California,Modesto,1900 Vernon Ave,1,3,"1900 Vernon Ave, Modesto California"
4,1581103,"December 27, 2019",Texas,Houston,500 Smart St,2,7,"500 Smart St, Houston Texas"


In [9]:
result.to_csv("stage1_transform.csv")

## Geopy

In [10]:
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim

locator = Nominatim(user_agent="myGeocoder")

geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
geocode = locator.geocode

In [11]:
def find_location(row):
    
    place = row['full_address']
    
    location = locator.geocode(place)
    
    if location != None:
        return location.latitude, location.longitude
    else:
        return "Not Found", "Not Found"

In [12]:
result[['lat', 'long']] = result.apply(find_location, axis="columns", result_type="expand")

In [13]:
result.head()

,incident_id,incident_date,state,city_county,street_address,killed,injured,full_address,lat,long
0,1582816,"December 29, 2019",Illinois,Danville,1803 Westview Ave,0,5,"1803 Westview Ave, Danville Illinois",40.15553,-87.614715
1,1582197,"December 29, 2019",New York,Buffalo,50 Henrietta Ave,1,3,"50 Henrietta Ave, Buffalo New York",42.956321,-78.899348
2,1583444,"December 29, 2019",California,Ceres,800 Allacante Dr,0,5,"800 Allacante Dr, Ceres California",37.590897,-120.981631
3,1581590,"December 27, 2019",California,Modesto,1900 Vernon Ave,1,3,"1900 Vernon Ave, Modesto California",37.614475,-121.027922
4,1581103,"December 27, 2019",Texas,Houston,500 Smart St,2,7,"500 Smart St, Houston Texas",29.959913,-95.562268


In [14]:
result.to_csv("stage2_transform_geopy.csv")

## Google Geolocator

In [15]:
just_nulls_df = pd.read_csv("stage2_transform_geopy.csv")

In [16]:
just_nulls_df = just_nulls_df.loc[just_nulls_df["lat"] == "Not Found",:]

In [17]:
just_nulls_df

,Unnamed: 0,incident_id,incident_date,state,city_county,street_address,killed,injured,full_address,lat,long
5,5,1581476,"December 27, 2019",Georgia,Kennesaw,1575 Ridenour Pkwy NW,0,4,"1575 Ridenour Pkwy NW, Kennesaw Georgia",Not Found,Not Found
14,14,1577688,"December 22, 2019",Minnesota,Minneapolis (Spring Lake Park),8407 Plaza Blvd NE,1,7,"8407 Plaza Blvd NE, Minneapolis (Spring Lake P...",Not Found,Not Found
32,32,1562685,"December 1, 2019",Michigan,Kalamazoo,6300 Proctor St,1,3,"6300 Proctor St, Kalamazoo Michigan",Not Found,Not Found
34,34,1561398,"November 30, 2019",Arkansas,Hensley,6500 E Sardis Rd,0,5,"6500 E Sardis Rd, Hensley Arkansas",Not Found,Not Found
36,36,1559257,"November 27, 2019",New York,Bronx,E 153rd St and Courtlandt Ave,0,5,"E 153rd St and Courtlandt Ave, Bronx New York",Not Found,Not Found
...,...,...,...,...,...,...,...,...,...,...,...
1534,506,1897878,"January 9, 2021",Texas,Houston,5828 Martin Luther King Blvd,1,3,"5828 Martin Luther King Blvd, Houston Texas",Not Found,Not Found
1537,509,1896252,"January 6, 2021",Virginia,Manassas,10010 Ellis Rd,3,3,"10010 Ellis Rd, Manassas Virginia",Not Found,Not Found
1539,511,1894979,"January 4, 2021",Florida,Tampa,1225 S 78th St,0,5,"1225 S 78th St, Tampa Florida",Not Found,Not Found
1540,512,1894941,"January 4, 2021",Florida,Fort Lauderdale (Lauderhill),1828 NW 38th Ave,0,7,"1828 NW 38th Ave, Fort Lauderdale (Lauderhill)...",Not Found,Not Found


In [18]:
just_nulls_df = just_nulls_df.reset_index()
just_nulls_df = just_nulls_df.drop(['Unnamed: 0'], axis=1)

In [19]:
def find_geocode(row):
    
    place = row['full_address']
    place = place.replace(" ", "+")
    
    #gkey = ""

    # Build the endpoint URL
    target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={place}&key={gkey}"
    

    try:
        geo_data = requests.get(target_url).json()
        lat = geo_data["results"][0]["geometry"]["location"]["lat"]
        lng = geo_data["results"][0]["geometry"]["location"]["lng"]
        
        if lat != None:
            return lat, lng
        else:
            return "Not Found", "Not Found"
    
    except:
        return "Not Found", "Not Found"

In [20]:
just_nulls_df[['lat', 'long']] = just_nulls_df.apply(find_geocode, axis="columns", result_type="expand")

In [21]:
just_nulls_df.head()

,index,incident_id,incident_date,state,city_county,street_address,killed,injured,full_address,lat,long
0,5,1581476,"December 27, 2019",Georgia,Kennesaw,1575 Ridenour Pkwy NW,0,4,"1575 Ridenour Pkwy NW, Kennesaw Georgia",33.991723,-84.588958
1,14,1577688,"December 22, 2019",Minnesota,Minneapolis (Spring Lake Park),8407 Plaza Blvd NE,1,7,"8407 Plaza Blvd NE, Minneapolis (Spring Lake P...",45.122426,-93.234282
2,32,1562685,"December 1, 2019",Michigan,Kalamazoo,6300 Proctor St,1,3,"6300 Proctor St, Kalamazoo Michigan",42.285262,-85.506099
3,34,1561398,"November 30, 2019",Arkansas,Hensley,6500 E Sardis Rd,0,5,"6500 E Sardis Rd, Hensley Arkansas",34.544067,-92.355177
4,36,1559257,"November 27, 2019",New York,Bronx,E 153rd St and Courtlandt Ave,0,5,"E 153rd St and Courtlandt Ave, Bronx New York",40.819372,-73.918284


In [22]:
new_df = just_nulls_df.loc[just_nulls_df["lat"] != "Not Found",:]

In [23]:
new_df = new_df.drop(['index'], axis=1)

In [24]:
new_df.to_csv("stage3_transform.csv")

### Merge the dataframes

In [25]:
first_file = pd.read_csv("stage2_transform_geopy.csv")
second_File = pd.read_csv("stage3_transform.csv")

In [26]:
cleaned_df = first_file.loc[first_file["lat"] != "Not Found"]

In [27]:
cleaned_df = cleaned_df.drop(['Unnamed: 0'], axis=1)

In [28]:
second_File = second_File.drop(['Unnamed: 0'], axis=1)

In [29]:
frames2 = [cleaned_df, second_File]

complete_df = pd.concat(frames2)

In [30]:
complete_df = complete_df.dropna(how='any')

In [31]:
complete_df.head()

,incident_id,incident_date,state,city_county,street_address,killed,injured,full_address,lat,long
0,1582816,"December 29, 2019",Illinois,Danville,1803 Westview Ave,0,5,"1803 Westview Ave, Danville Illinois",40.1555301092308,-87.61471508589744
1,1582197,"December 29, 2019",New York,Buffalo,50 Henrietta Ave,1,3,"50 Henrietta Ave, Buffalo New York",42.9563206122449,-78.89934789795919
2,1583444,"December 29, 2019",California,Ceres,800 Allacante Dr,0,5,"800 Allacante Dr, Ceres California",37.590897,-120.981631
3,1581590,"December 27, 2019",California,Modesto,1900 Vernon Ave,1,3,"1900 Vernon Ave, Modesto California",37.614474926174495,-121.02792171140939
4,1581103,"December 27, 2019",Texas,Houston,500 Smart St,2,7,"500 Smart St, Houston Texas",29.95991275,-95.56226793134292


In [32]:
complete_df.to_csv("completed_data.csv")

# Load

In [33]:
from sqlalchemy import create_engine

rds_connection_string = "postgres:postgres@localhost:5432/mass_shootings_project3"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Create Table
<br>
<li> Run once </li>

In [34]:
create_table = engine.execute('CREATE TABLE "Mass_Shootings" ( "incident_id" varchar   NOT NULL, "incident_date" varchar   NOT NULL,"state" varchar   NOT NULL,"city_county" varchar   NOT NULL,"street_address" varchar   NOT NULL,"killed" int   NOT NULL,"injured" int   NOT NULL,"full_address" varchar   NOT NULL,"lat" float   NOT NULL,"long" float   NOT NULL);')

In [35]:
engine.table_names()

<ipython-input-35-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['Mass_Shootings']

### Write to database

In [36]:
file_to_database = pd.read_csv("completed_data.csv")

In [37]:
file_to_database = file_to_database.drop(['Unnamed: 0'], axis=1)

In [38]:
file_to_database.to_sql(name='Mass_Shootings', con=engine, if_exists='append', index=False)